# TRYING GRAPHRAG with NEO4J

![HLD.png](https://i.ibb.co/LPcqkKg/Screenshot-2024-12-19-190838.png)

In [1]:
%pip install --upgrade --quiet  langchain langchain-experimental langchain-openai langchain-neo4j neo4j wikipedia langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [9]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
import os

diffbot_api_key = os.getenv('DIFFBOT_API_KEY')
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

## Creating and storing in neo4j graphdb

In [25]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.document_loaders import PyPDFLoader

# query = "Warren Buffett"
# raw_documents = WikipediaLoader(query=query).load()
raw_documents = PyPDFLoader(file_path="../Docs/swebokv3.pdf").load()
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

In [32]:
from langchain_neo4j import Neo4jGraph

url = os.getenv('NEO4J_URL')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph = Neo4jGraph(url=url, username=username, password=password)

In [27]:
graph_documents

[GraphDocument(nodes=[], relationships=[], source=Document(metadata={'source': '../Docs/swebokv3.pdf', 'page': 0}, page_content='')),
 GraphDocument(nodes=[], relationships=[], source=Document(metadata={'source': '../Docs/swebokv3.pdf', 'page': 1}, page_content='Guide to the Software Engineering  \nBody of Knowledge\nVersion 3.0\nSWEBOK\n®\nA Project of the IEEE Computer Society')),
 GraphDocument(nodes=[], relationships=[], source=Document(metadata={'source': '../Docs/swebokv3.pdf', 'page': 2}, page_content='Guide to the Software Engineering  \nBody of Knowledge\nVersion 3.0\nEditors\nPierre Bourque, École de technologie supérieure (ÉTS)\nRichard E. (Dick) Fairley, Software and Systems Engineering Associates (S2EA)')),
 GraphDocument(nodes=[Node(id='Kate Guillemette', type='Person', properties={'name': 'Kate Guillemette', 'positionHeld': 'Product Development Editor'}), Node(id='http://www.linkedin.com/in/jenniezhumai', type='Person', properties={'name': 'Jennie Zhu-Mai', 'positionHeld

In [28]:
graph.add_graph_documents(graph_documents)

In [33]:
graph.refresh_schema()


In [36]:
from langchain_neo4j import GraphCypherQAChain
from langchain_ollama.chat_models import ChatOllama

chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOllama(model='llama3.2:1b'),
    qa_llm=ChatOllama(model='llama3.2:1b'),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

In [37]:
chain.run("What is software?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:EDUCATED_AT]->(o:Organization) WHERE o.name = "software" RETURN p, o LIMIT 1
Full Context:
[]

> Finished chain.


'I\'m not aware of any specific definition or classification of "software." However, it is generally understood to refer to a set of instructions or programs designed to perform specific tasks or provide services. This can include operating systems, applications, frameworks, and other types of digital products that enable users to interact with computers or other devices.'